In [1]:
from osrs_GE_estimators import ARIMAHybridClassifier as AHC, ARIMAHybridModel as AHM
import osrs_GE

import numpy as np
import pandas as pd

from pmdarima.arima import auto_arima
from sklearn.ensemble import RandomForestClassifier

from statsmodels.tsa.arima.model import ARIMA

In [2]:
item_id = 11284 # dragonfire shield
freq = '6h' # 6 hour data

base_df = osrs_GE.load_item_ts(item_id,freq)

# work with a copy of the base data set
# in case we need the original for a recomputation?
df = base_df.copy()

# compute VWAP
osrs_GE.compute_VWAP(df)

# compute simple returns
df['simpRet_y'] = (df['VWAP'].shift(-1)/df['VWAP']) - 1

# generate features
osrs_GE.compute_TA_features(df,lagged_rets = 0,RSI_window=5,MACD_short=4,MACD_long=8,SMA_window=5,inplace=True)


df.dropna(inplace=True)

train_df = df.iloc[:int(len(df)*.7)]

In [3]:
arima_res = auto_arima(train_df['simpRet_y'])
print("Auto ARIMA order: ",arima_res.order)

n_estimators = 5
max_depth = 80

rfc = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth,
                                   random_state = 8)

Auto ARIMA order:  (2, 0, 2)


In [7]:
ahc = AHC(arima_order = arima_res.order, ml_model=rfc,pos_threshold=.005,neg_threshold=-.0075)

In [9]:
ahc.fit(train_df.drop('simpRet_y',axis=1),train_df['simpRet_y'])

ARIMAHybridClassifier(arima_order=(2, 0, 2),
                      ml_model=RandomForestClassifier(max_depth=80,
                                                      n_estimators=5,
                                                      random_state=8),
                      neg_threshold=-0.0075, pos_threshold=0.005)

In [29]:
tmp = train_df.drop('simpRet_y',axis=1).iloc[-5:,:]
tmp2= ahc._create_lagged_features(ahc.arima_model.resid,forecast_mode=True,num_steps=1)

,2024-10-05 18:00:00
VWAP_RSI5,44.181650
OI,83.000000
OI_ratio,0.107932
SpreadPct,0.011472
CM,0.952201
VWAP/ema4,0.997235
VWAP/ema8,0.992155
VWAP_nMACD_4_8,-0.005134


In [49]:
ahc.predict(train_df.drop('simpRet_y',axis=1))

C:\Users\Cody\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


ValueError: Length mismatch: Expected axis has 462 elements, new values have 463 elements

In [ ]:
ahc.

In [13]:
train_df.drop('simpRet_y',axis=1).iloc[-5:,:]

,VWAP_RSI5,OI,OI_ratio,SpreadPct,CM,VWAP/ema4,VWAP/ema8,VWAP_nMACD_4_8
date,,,,,,,,
2024-10-05 06:00:00,31.074876,33,0.089918,0.016414,0.541659,0.997674,0.988624,-0.009176
2024-10-05 12:00:00,31.839050,52,0.085246,0.011646,0.605579,1.005319,0.999802,-0.005489
2024-10-05 18:00:00,44.181650,83,0.107932,0.011472,0.952201,0.997235,0.992155,-0.005134
2024-10-06 00:00:00,46.422283,14,0.025547,0.012026,0.168360,0.999847,0.995836,-0.004028
2024-10-06 06:00:00,61.222481,70,0.200000,0.012579,0.880520,0.999494,0.996223,-0.003285
